In [0]:
-- use redox.hls_webinar_fy25q4;
use fhir_workshop.synthea;

with eg as (
  SELECT
    "post" as http_method
    ,"DiagnosticReport" as resource
    ,"_search" as action
    ,"subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7" as data
)
,response as (
  SELECT
    try_parse_json(redox_request(
    http_method
    ,resource
    ,action
    ,data
  )) as response
FROM 
  eg
)
SELECT
  entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.based_on
  ,entry.value:resource.code
  ,entry.value:resource.encounter.reference::string as encounter_url
  ,entry.value:resource.id::string as diagnostic_report_id
  ,entry.value:resource.issued::timestamp as issued
  ,entry.value:resource.meta
  ,presentedForm.value:contentType::string as content_type
  ,presentedForm.value:creation::timestamp as creation
  ,unbase64(presentedForm.value:data::string)::string as diagnostic_report
FROM 
  response 
  ,LATERAL variant_explode(response:entry) as entry
  ,LATERAL variant_explode(entry.value:resource.presentedForm) as presentedForm
;